In [45]:
from pylabel import importer, exporter, dataset
import numpy as np
from sklearn.model_selection import train_test_split

In [37]:
voc_path = '/home/gridsan/manderson/ovdsat/data/DIOR'

In [38]:
mydataset = importer.ImportVOC(path = voc_path+'/Annotations', path_to_images=voc_path+'/JPEGImages', name='DIOR')


Importing VOC files...: 100%|██████████| 17403/17403 [00:09<00:00, 1763.82it/s]


In [39]:
print(mydataset.df)

                                              img_folder img_filename  \
id                                                                      
0      /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    17635.jpg   
1      /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    17635.jpg   
2      /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    17635.jpg   
3      /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    12474.jpg   
4      /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    12474.jpg   
...                                                  ...          ...   
38315  /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    04219.jpg   
38316  /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    04219.jpg   
38317  /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    07822.jpg   
38318  /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    07822.jpg   
38319  /home/gridsan/manderson/ovdsat/data/DIOR/JPEGI...    07822.jpg   

      img_path  img_id  img_width  img_height img_

In [51]:
print(mydataset.df['img_id'].nunique())

17402


#### Generate splits for N=5,10,30, finetune, and validation

In [43]:
save_dir = '/home/gridsan/manderson/ovdsat/data/DIOR'

# Create N splits
df_remaining = mydataset.df
for N in [5, 10, 30]:
    cat_id = df_remaining['cat_id']
    img_ids = []
    random_indices = {}
    for value, group in cat_id.groupby(cat_id):
        indices = group.index
        random_sample = np.random.choice(indices, N, replace=False)
        random_indices[value] = random_sample.tolist()
        img_ids.extend(df_remaining.loc[random_sample, 'img_id'].tolist())

    df_N = df_remaining[df_remaining['img_id'].isin(img_ids)]
    df_N = df_N.reset_index(drop=True)
    print(len(df_N))
    
    df_remaining = df_remaining[~df_remaining['img_id'].isin(img_ids)]
    print(len(df_remaining))
    
    pylabel_df_N = dataset.Dataset(df_N)
    exp = exporter.Export(pylabel_df_N)
    exp.ExportToCoco(output_path=f'{save_dir}/train_coco_subset_N{N}.json')

#Split remaining into val finetune (10% of remaining) and val (90% of remaining)
ft_percent = 0.1
img_ids = df_remaining['img_id'].unique()

# Split into train and test
val_ids, ft_ids = train_test_split(img_ids, test_size=ft_percent, random_state=42)

val_df = df_remaining[df_remaining['img_id'].isin(val_ids)]
ft_df = df_remaining[df_remaining['img_id'].isin(ft_ids)]

print(len(val_df))
print(len(ft_df))

val_df = val_df.reset_index(drop=True)
pylabel_val_df = dataset.Dataset(val_df)
exp = exporter.Export(pylabel_val_df)
exp.ExportToCoco(output_path=f'{save_dir}/val_coco.json')

ft_df = ft_df.reset_index(drop=True)
pylabel_ft_df = dataset.Dataset(ft_df)
exp = exporter.Export(pylabel_ft_df)
exp.ExportToCoco(output_path=f'{save_dir}/train_coco_finetune_val.json')

/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


{'0': [24437, 11862, 33420, 19352, 22677], '1': [1410, 34215, 16063, 21902, 14645], '10': [27693, 17126, 35091, 21272, 12870], '11': [30620, 31643, 27789, 37840, 7488], '12': [34610, 32961, 11876, 10062, 23534], '13': [15777, 10635, 11140, 4513, 16848], '14': [7975, 770, 28242, 12430, 10789], '15': [6577, 26815, 20242, 10043, 22722], '16': [24909, 3817, 35808, 13052, 14436], '17': [409, 2012, 38261, 23603, 17871], '18': [17372, 37357, 13899, 14268, 25784], '19': [17149, 13124, 30892, 27865, 16242], '2': [2411, 18257, 4413, 8465, 1487], '3': [29313, 33134, 18579, 10431, 10428], '4': [25580, 9615, 5188, 37108, 11796], '5': [36068, 17517, 1175, 33422, 5000], '6': [16081, 11192, 7248, 3160, 26632], '7': [3814, 29774, 195, 16323, 37284], '8': [6495, 24744, 38095, 25713, 26127], '9': [9846, 4346, 31249, 7061, 2923]}
100
[11095, 5362, 15172, 8755, 10298, 629, 15534, 7265, 9951, 6628, 12593, 7732, 15923, 9658, 5810, 13899, 14360, 12642, 17173, 3366, 15712, 14959, 5366, 4562, 10665, 7130, 4822,

Exporting to COCO file...: 100%|██████████| 270/270 [00:00<00:00, 2184.96it/s]
/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


{'0': [25896, 18360, 38029, 22344, 30622, 471, 10508, 3244, 24772, 10332], '1': [15046, 4927, 8043, 6118, 36136, 13333, 8160, 28648, 32006, 23377], '10': [18859, 12353, 9038, 24252, 3485, 12657, 26095, 2358, 32601, 16029], '11': [8597, 36716, 35013, 14517, 1727, 11125, 3991, 30967, 14887, 13784], '12': [26870, 33460, 28565, 13740, 32083, 34538, 32009, 38124, 26454, 11736], '13': [27015, 11610, 31684, 3135, 16481, 15863, 37451, 15290, 23364, 20662], '14': [34439, 37509, 14853, 28881, 18835, 26562, 21431, 23284, 3630, 10589], '15': [2421, 11081, 32081, 25635, 4820, 6678, 14128, 12769, 28504, 31881], '16': [34280, 4915, 16209, 15822, 22757, 2076, 37375, 13863, 16402, 21338], '17': [8219, 25623, 36393, 32347, 27672, 1697, 13321, 5603, 32664, 20743], '18': [9765, 30646, 32353, 5894, 22433, 37306, 28268, 18156, 33170, 31045], '19': [37524, 30964, 14340, 8599, 24986, 25668, 38077, 7323, 19934, 5516], '2': [19017, 23963, 24359, 7040, 2786, 13699, 2782, 26392, 7923, 3560], '3': [28097, 28372, 4

Exporting to COCO file...: 100%|██████████| 573/573 [00:00<00:00, 2921.03it/s]
/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


{'0': [21516, 30491, 3439, 33204, 4329, 3074, 24232, 1104, 6642, 18080, 6006, 4139, 1848, 25157, 35687, 22416, 20389, 9085, 29545, 12178, 24675, 3390, 9480, 8786, 19695, 2214, 33532, 24, 23479, 18460], '1': [17130, 14805, 14178, 2859, 2766, 9065, 32761, 26216, 3497, 8711, 4221, 21041, 14060, 3872, 11704, 25166, 21648, 31234, 6898, 31766, 29564, 21761, 11161, 20010, 8845, 13558, 25108, 7283, 12048, 14184], '10': [10948, 28981, 37811, 21204, 36092, 37204, 30641, 26430, 13094, 22, 29874, 33810, 9159, 36902, 13164, 28926, 18206, 8966, 32995, 21773, 12563, 26877, 37610, 11980, 33101, 14414, 605, 35941, 21638, 21937], '11': [29661, 26213, 27352, 27068, 8270, 33339, 35528, 34004, 16215, 8072, 5503, 2806, 1322, 10552, 36524, 11788, 1751, 8200, 25035, 26424, 11345, 24432, 31632, 11344, 32836, 3301, 18255, 7981, 10846, 29752], '12': [33097, 28587, 15764, 36610, 3333, 981, 27999, 5337, 4898, 3432, 33365, 37231, 37722, 8119, 38233, 6584, 1022, 8773, 17983, 527, 19144, 6582, 6928, 21898, 21363, 306

Exporting to COCO file...: 100%|██████████| 1725/1725 [00:00<00:00, 2270.14it/s]
